In [27]:
import json
import pandas as pd

In [28]:
def load_matrix(matrix_name, matrix_origin='matrices.xlsx'):
    matrix = f'{matrix_name} %{matrix_name}'.split(' ')
    return {matrix: pd.read_excel(matrix_origin, matrix, header=None) for matrix in matrix}

def matrix_intercalation(matrix_x, matrix_name):
    cols = []
    for i in range(4):
        cols.append(matrix_x[matrix_name].iloc[:,i])
        cols.append(matrix_x[f'%{matrix_name}'].iloc[:,i])

    matrix = pd.concat(cols, axis=1)
    matrix.columns = 'r1 w1 r2 w2 r3 w3 r4 w4'.split(' ')
    return matrix

def matrix_weight_correction(weight, matrix, matrix_name):
    matrix[f'%{matrix_name}'] = matrix[f'%{matrix_name}'] * weight

In [29]:
def matrix_creation(exercise_class, exercise_weight):

    exercise_class = 'A' if exercise_class == 1 else 'B'

    matrix = load_matrix(exercise_class)
    matrix_weight_correction(exercise_weight, matrix, exercise_class)
    matrix = matrix_intercalation(matrix, exercise_class)
    return matrix

In [ ]:
def exercise_extended_calendar_matrix(day, file='rms.xlsx',):
    
    exercise_extended_calendar = dict()
    
    exercises_data = pd.read_excel(file, day, header=None)
    
    for _, row in exercises_data.iterrows():
        exercise_name, exercise_rm, exercise_class = row
        exercise_extended_calendar.update({exercise_name: matrix_creation(exercise_class, exercise_rm)})

    exercises_names = exercises_data[0]

    return exercise_extended_calendar, exercises_names

def create_json(exercise_extended_calendar, exercises_names, filename="routine.json"):
    training_day = {}
    for exercise in exercises_names:
        training_day[exercise] = exercise_extended_calendar[exercise].iloc[0, :]

    training_day_json = {k: v.to_dict() for k, v in training_day.items()}

    # Guardar en archivo
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(training_day_json, f, indent=4, ensure_ascii=False)

    print(f"Archivo JSON guardado en {filename}")

In [31]:
def muscleapp_json(day):
    exercise_extended_calendar, exercises_names = exercise_extended_calendar_matrix(day)
    create_json(exercise_extended_calendar, exercises_names)

In [33]:
muscleapp_json('D')

Archivo JSON guardado en routine.json
